In [ ]:
# Rode o código se precisar instalar o snscrape
%pip install git+https://github.com/JustAnotherArchivist/snscrape.git

# Rode esse código se precisar instalar o Pandas
# !pip install pandas

In [2]:
# Imports
import snscrape.modules.twitter as sntwitter
import pandas as pd
import datetime as dt


In [3]:
def ScrapTweets(max_tweets: int, search_string: str):
    # Definindo o máximo de tweets
    maxTweets = max_tweets

    # Criando a lista para salvar os dados
    tweets_list = []

    # Usando o TwitterSearchScraper para fazer o scrape dos dados data e salvar em uma lista
    for i, tweet in enumerate(
        sntwitter.TwitterSearchScraper(search_string).get_items()
    ):
        if i > maxTweets:
            break
        tweets_list.append(
            [
                tweet.date,
                tweet.replyCount,
                tweet.retweetCount,
                tweet.likeCount,
                tweet.place,
                tweet.sourceLabel,
                tweet.rawContent,
                tweet.user.username,
                tweet.user.followersCount,
                tweet.user.friendsCount,
                tweet.user.created,
                tweet.user.verified,
                tweet.user.statusesCount,
                tweet.user.favouritesCount,
                tweet.user.listedCount,
                tweet.user.mediaCount,
            ]
        )
    # Criando um dataframe com os dados
    tweets_df = pd.DataFrame(
        tweets_list,
        columns=[
            "DataHora",
            "ReplyCount",
            "Retweet Cont",
            "LikeCount",
            "Local",
            "Source",
            "Texto",
            "Usuario",
            "NumSeguidores",
            "NumAmigos",
            "UsuarioCriado",
            "UsuarioVerificado",
            "StatusCount",
            "favouritesCount",
            "listedCount",
            "mediaCount",
        ],
    )
    # Adicionando a coluna de data, mês e hora
    tweets_df["dia"] = tweets_df.DataHora.dt.day
    tweets_df["mes"] = tweets_df.DataHora.dt.month
    tweets_df["ano"] = tweets_df.DataHora.dt.year

    return tweets_df


In [4]:
def FormatToIRAMUTEQ(tweets: pd.DataFrame):
    # Criando um dicionário a partir do dataframe
    tweets_dict = tweets.to_dict()

    with open("corpus.txt", "w", encoding="utf-8") as corpus:
        for i in range(len(tweets_dict["DataHora"])):
            corpus.write("**** ")
            corpus.write(f'*ano_{tweets_dict["ano"][i]} ')
            corpus.write(f'*mes_{tweets_dict["mes"][i]} ')
            corpus.write(f'*dia_{tweets_dict["dia"][i]} ')
            corpus.write(f'*usuario_{tweets_dict["Usuario"][i]} ')
            corpus.write(f'*fonte_{tweets_dict["Source"][1].replace(" ", "_")}\n')
            corpus.write(tweets_dict["Texto"][i].replace("\n", " ").replace("*", ""))
            corpus.write("\n\n")


In [5]:
# Na string de busca você pode usar ou omitir os marcadores from e until
# Exemplo:  'rockinrio from:anitta until:2020-07-31'

tweets = ScrapTweets(100, "COVID vacina")

tweets.head()


,DataHora,ReplyCount,Retweet Cont,LikeCount,Local,Source,Texto,Usuario,NumSeguidores,NumAmigos,UsuarioCriado,UsuarioVerificado,StatusCount,favouritesCount,listedCount,mediaCount,dia,mes,ano
0,2022-09-06 01:41:51+00:00,0,0,0,None,Twitter for iPhone,Repetindo: hora de @simonetebetbr convidar o @...,AnaCandocha,1706,1557,2010-05-26 16:40:10+00:00,False,118291,8418,44,1007,6,9,2022
1,2022-09-06 01:41:37+00:00,0,0,0,None,tribunadosertao,Covid-19: vacina brasileira mira variantes e f...,tribunasertao,1450,308,2012-04-11 04:39:36+00:00,False,227708,119,16,56881,6,9,2022
2,2022-09-06 01:39:05+00:00,0,0,0,None,Twitter for iPhone,@folha Vacina da COVID tirou a credibilidade d...,CostaCarvey,18,28,2012-08-08 19:42:59+00:00,False,2378,1940,0,1035,6,9,2022
3,2022-09-06 01:32:19+00:00,0,0,1,None,Twitter for Android,@Leopoldo77 @nomeobjeto @Euclides_crf quem não...,brwndxn,1305,2397,2014-10-02 15:54:39+00:00,False,54290,162996,6,4220,6,9,2022
4,2022-09-06 01:31:37+00:00,0,0,9,None,Twitter for Android,@desmentindobozo O meu pai morreu de covid . D...,CarlaFioori,75,307,2022-01-03 20:21:56+00:00,False,69,418,0,1,6,9,2022


In [15]:
# Exportar dataframe para CSV
tweets.to_csv("text-query-tweets.csv", sep=",", index=False)

# Criar corpus para IRAMUTEQ
FormatToIRAMUTEQ(tweets)